# preparacao do dataset

# importando bibliotecas para o tratamento

In [46]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import unicodedata
import emoji

nltk.download('stopwords', quiet=True)

True

In [25]:
review = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")
review.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


# verificando e removendo valores nulos nas colunas principais

In [27]:
print(review[['review_id','review_score','review_comment_message']].isnull().sum())

review_id                     0
review_score                  0
review_comment_message    58247
dtype: int64


In [29]:
review = review.dropna(subset=['review_id','review_score','review_comment_message']).copy()
print('formato após remoção de nulos:', review.shape)

formato após remoção de nulos: (40977, 7)


# removendo duplicatas exatas no texto de review

In [31]:
dup_count = review.duplicated(subset=['review_comment_message']).sum()
print('Duplicatas encontradas:', dup_count)

Duplicatas encontradas: 4818


In [33]:
review = review.drop_duplicates(subset=['review_comment_message']).reset_index(drop=True)
print('formato após remover duplicatas:', review.shape)

formato após remover duplicatas: (36159, 7)


# removendo score = 3

In [ ]:
review = review[review['review_score'] != 3].copy()
print('formato após remover notas 3:', review.shape)
print(review['review_score'].value_counts())

Shape após remover notas 3: (32901, 7)
review_score
5    17369
1     8388
4     5093
2     2051
Name: count, dtype: int64


# criando a coluna sentiment

In [35]:
# Criar coluna 'sentiment'
review['sentiment'] = review['review_score'].apply(lambda x: 1 if x >= 4 else 0)

# Conferir distribuição
print(review['sentiment'].value_counts())

sentiment
1    22462
0    10439
Name: count, dtype: int64


# balanceando valores positivos e negativos

In [36]:
negativos = review[review['sentiment'] == 0]
positivos = review[review['sentiment'] == 1]

n_neg = len(negativos)
print(f'\nQuantidade de negativos: {n_neg}')
print(f'Quantidade de positivos antes do balanceamento: {len(positivos)}')


Quantidade de negativos: 10439
Quantidade de positivos antes do balanceamento: 22462


In [37]:
positivos_down = positivos.sample(n=n_neg, random_state=42)

review_balanced = (
    pd.concat([negativos, positivos_down])
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

print("\nDepois do balanceamento:")
print(review_balanced['sentiment'].value_counts())


Depois do balanceamento:
sentiment
1    10439
0    10439
Name: count, dtype: int64


# selecionando colunas importantes

In [39]:
review_final = review[['review_id','review_score','review_comment_message','sentiment']].copy()
review_final.head()

,review_id,review_score,review_comment_message,sentiment
0,e64fb393e7b32834bb789ff8bb30750e,5,Recebi bem antes do prazo estipulado.,1
1,f7c4243c7fe1938f181bec41a392bdeb,5,Parabéns lojas lannister adorei comprar pela I...,1
2,8670d52e15e00043ae7de4c01cc2fe06,4,aparelho eficiente. no site a marca do aparelh...,1
3,4b49719c8a200003f700d3d986ea1a19,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",1
4,3948b09f7c818e2d86c9a546758b2335,5,"Vendedor confiável, produto ok e entrega antes...",1


# tratamento do dataset

In [47]:
stopwords_pt = set(stopwords.words('portuguese'))

clean = review_final['review_comment_message'].astype(str)
clean = clean.str.lower()
clean = clean.apply(lambda txt: emoji.replace_emoji(txt, replace=' '))

In [48]:
clean = clean.apply(lambda txt: unicodedata.normalize('NFD', txt))
clean = clean.str.replace(r'[\u0300-\u036f]', '', regex=True)
clean = clean.str.replace(r'[^a-z\s]', ' ', regex=True)

In [49]:
clean = clean.str.split().apply(
    lambda tokens: [w for w in tokens if w not in stopwords_pt and len(w) > 1]
)
clean = clean.str.join(' ')

In [50]:
clean = clean.str.replace(r'\s+', ' ', regex=True).str.strip()

In [51]:
review_final['clean_review'] = clean
review_final[['review_comment_message','clean_review']].head()

,review_comment_message,clean_review
0,Recebi bem antes do prazo estipulado.,recebi bem antes prazo estipulado
1,Parabéns lojas lannister adorei comprar pela I...,parabens lojas lannister adorei comprar intern...
2,aparelho eficiente. no site a marca do aparelh...,aparelho eficiente site marca aparelho impress...
3,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",pouco travando valor ta boa
4,"Vendedor confiável, produto ok e entrega antes...",vendedor confiavel produto ok entrega antes prazo


# salvando o dataset processado